In [1]:
import jieba.posseg as pseg
import codecs
from gensim import corpora, models, similarities
import pandas as pd
import numpy as np
import matplotlib as mpl
import tensorflow as tf
import keras

%matplotlib inline

D:\program\Anaconda\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


In [2]:
stop_words = 'stopwords.txt'
stopwords = codecs.open(stop_words,'r',encoding='utf8').readlines()
stopwords = [ w.strip() for w in stopwords ]
stopwords

['!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '--',
 '.',
 '..',
 '...',
 '......',
 '...................',
 './',
 '.一',
 '.数',
 '.日',
 '/',
 '//',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 '://',
 '::',
 ';',
 '<',
 '=',
 '>',
 '>>',
 '?',
 '@',
 'A',
 'Lex',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 'exp',
 'sub',
 'sup',
 '|',
 '}',
 '~',
 '~~~~',
 '·',
 '×',
 '×××',
 'Δ',
 'Ψ',
 'γ',
 'μ',
 'φ',
 'φ．',
 'В',
 '—',
 '——',
 '———',
 '‘',
 '’',
 '’‘',
 '“',
 '”',
 '”，',
 '…',
 '……',
 '…………………………………………………③',
 '′∈',
 '′｜',
 '℃',
 'Ⅲ',
 '↑',
 '→',
 '∈［',
 '∪φ∈',
 '≈',
 '①',
 '②',
 '②ｃ',
 '③',
 '③］',
 '④',
 '⑤',
 '⑥',
 '⑦',
 '⑧',
 '⑨',
 '⑩',
 '──',
 '■',
 '▲',
 '',
 '、',
 '。',
 '〈',
 '〉',
 '《',
 '》',
 '》），',
 '」',
 '『',
 '』',
 '【',
 '】',
 '〔',
 '〕',
 '〕〔',
 '㈧',
 '一',
 '一.',
 '一一',
 '一下',
 '一个',
 '一些',
 '一何',
 '一切',
 '一则',
 '一则通过',
 '一天',
 '一定',
 '一方面',
 '一旦',
 '一时',
 '一来',
 '一样',
 '一次',
 '一片',
 '一番',
 '一直',
 '一致',
 '一般',
 '一起',
 '一转眼'

In [3]:
#停用词性 [标点符号、连词、助词、副词、介词、时语素、‘的’、数词、方位词、代词]
stop_flag = ['x', 'c', 'u','d', 'p', 't', 'uj', 'm', 'f', 'r']

In [4]:
def tokenization(text):
    result = []
    words = pseg.cut(text)
    for word, flag in words:
        if flag not in stop_flag and word not in stopwords:
            result.append(word)
    return result

In [5]:
df_record = pd.read_csv('admission_mr.csv')
df_record.head()

,ID,SYSTYPE,DATATYPE,PATIENT_ID,PATIENT_NO,INPATIENT_NO,INDEX2,PARENTNODENAME,NODENAME,NODEVALUE,...,OPERATOR_NAME,OPERATOR_TIME,NAME,UPDATED_ON,PROCESS_STATUS,SEQ_ID,HOSPITAL_CODE,OPERATOR_SYS_EMP_NO,DATA_VERSION,INP_PATIENT_NO
0,2199335,0,入院相关,0000545965,287627,ZY170000287627,曾*忠,入院病历\病史\既往史\手术外伤史,手术外伤史,NaN,...,何梦璋*,2012/12/27 星期四 12:56:41,再次或多次入院记录,05-6月 -18 02.53.06.000000 下午,NaN,31543,12440100455344205E,NaN,0,NaN
1,2199335,0,入院相关,0000545965,287627,ZY170000287627,曾*忠,入院病历\个人史\一般情况,一般情况,NaN,...,何梦璋*,2012/12/27 星期四 12:56:41,再次或多次入院记录,05-6月 -18 02.53.06.000000 下午,NaN,31544,12440100455344205E,NaN,0,NaN
2,2199339,0,入院相关,0000545965,287627,ZY170000287627,曾*忠,入院病历\体格检查一般情况\体位,体位,自主,...,何梦璋*,2012/12/27 星期四 12:59:44,再次或多次入院记录,05-6月 -18 02.53.06.000000 下午,NaN,31040,12440100455344205E,NaN,0,NaN
3,2199339,0,入院相关,0000545965,287627,ZY170000287627,曾*忠,入院病历\体格检查头面部\口唇,口唇,正常,...,何梦璋*,2012/12/27 星期四 12:59:44,再次或多次入院记录,05-6月 -18 02.53.06.000000 下午,NaN,31041,12440100455344205E,NaN,0,NaN
4,2199339,0,入院相关,0000545965,287627,ZY170000287627,曾*忠,入院病历\体格检查颈部\甲状腺,甲状腺,未及,...,何梦璋*,2012/12/27 星期四 12:59:44,再次或多次入院记录,05-6月 -18 02.53.06.000000 下午,NaN,31042,12440100455344205E,NaN,0,NaN


In [6]:
df_record_patient_describe = df_record[df_record['NODENAME'] == '主诉']
print(len(df_record_patient_describe))
df_record_patient_describe.head()

1768


,ID,SYSTYPE,DATATYPE,PATIENT_ID,PATIENT_NO,INPATIENT_NO,INDEX2,PARENTNODENAME,NODENAME,NODEVALUE,...,OPERATOR_NAME,OPERATOR_TIME,NAME,UPDATED_ON,PROCESS_STATUS,SEQ_ID,HOSPITAL_CODE,OPERATOR_SYS_EMP_NO,DATA_VERSION,INP_PATIENT_NO
109,2199339,0,入院相关,0000545965,287627,ZY170000287627,曾*忠,入院病历\病史\主诉,主诉,反复咳嗽、咳痰20多年，气促4年，加重４天。,...,何梦璋*,2012/12/27 星期四 13:03:05,再次或多次入院记录,05-6月 -18 02.53.06.000000 下午,NaN,31525,12440100455344205E,NaN,0,NaN
276,2318913,0,入院相关,0080324728,365305,ZY010000365305,黄*强,入院病历\病史\主诉,主诉,发热、咳嗽、气促1周。,...,梁增伟*,2013/2/28 星期四 16:41:03,入院记录,05-6月 -18 02.53.06.000000 下午,NaN,86785,12440100455344205E,NaN,0,NaN
316,2294384,0,入院相关,0000749211,321757,ZY060000321757,陈*成,入院病历\病史\主诉,主诉,咳嗽咳痰气促2年，加重1周。,...,叶枫**,2013/2/16 星期六 16:49:07,入院记录,05-6月 -18 02.53.06.000000 下午,NaN,89458,12440100455344205E,NaN,0,NaN
354,2329166,0,入院相关,0000033445,280478,ZY110000280478,吴*伙,入院病历\病史\主诉,主诉,反复咳嗽、咳痰伴气促7年，加重10天,...,李振球,2013/3/5 星期二 16:38:59,再次或多次入院记录,05-6月 -18 02.53.06.000000 下午,NaN,104334,12440100455344205E,NaN,0,NaN
417,2333860,0,入院相关,0080326996,365630,ZY010000365630,张*焕,入院病历\病史\主诉,主诉,右胸疼痛2月余,...,李树本,2013/3/7 星期四 15:20:27,入院记录,05-6月 -18 02.53.06.000000 下午,NaN,103667,12440100455344205E,NaN,0,NaN


In [7]:
df_patient_describe = df_record_patient_describe[['INPATIENT_NO','NODEVALUE']]
df_patient_describe.head()

,INPATIENT_NO,NODEVALUE
109,ZY170000287627,反复咳嗽、咳痰20多年，气促4年，加重４天。
276,ZY010000365305,发热、咳嗽、气促1周。
316,ZY060000321757,咳嗽咳痰气促2年，加重1周。
354,ZY110000280478,反复咳嗽、咳痰伴气促7年，加重10天
417,ZY010000365630,右胸疼痛2月余


In [8]:
corpus = []
for each in df_patient_describe['NODEVALUE']:
    text = str(each)
    corpus.append(tokenization(text))
print(len(corpus))

corpus

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\Acer\AppData\Local\Temp\jieba.cache
Loading model cost 1.062 seconds.
Prefix dict has been built succesfully.


1768


[['反复', '咳嗽', '咳痰', '气促', '加重', '天'],
 ['发热', '咳嗽', '气促', '周'],
 ['咳嗽', '咳痰', '气促', '加重', '周'],
 ['反复', '咳嗽', '咳痰', '伴', '气促', '加重', '天'],
 ['右胸', '疼痛', '月余'],
 ['咳嗽', '气促', '喘息', '加重', '伴', '发热', '天'],
 ['反复', '咳嗽', '咳痰', '伴', '气促', '加重', '天'],
 ['咳嗽', '咳痰', '气促', '加重'],
 ['反复', '咳嗽', '咳痰', '气促', '加重', '天'],
 ['nan'],
 ['反复', '咳嗽', '咳痰', '气促', '喘息', '伴', '胸痛', '天'],
 ['胸闷', '气促'],
 ['反复', '咳嗽', '咳痰', '气促', '加重', '天'],
 ['反复', '咳嗽', '咯痰', '加重', '周'],
 ['反复', '咳嗽', '咳痰', '气促', '加重', '伴', '发热', '天'],
 ['反复', '胸痛', '气促', '月余'],
 ['反复', '咳嗽', '咳痰', '气促', '加重', '天'],
 ['反复', '咳嗽', '咳痰', '气促', '加重'],
 ['活动', '气促', '月余', '加重', '周'],
 ['活动', '气促', '月余', '加重', '周'],
 ['反复', '咳嗽', '咳痰', '气促', '加重', '天'],
 ['反复', '咳嗽', '咳痰', '气促', '加重', '天'],
 ['反复', '咳嗽', '咳痰', '伴', '气促', '发', '周'],
 ['反复', '咳嗽', '气促', '发', '周'],
 ['反复', '咳嗽', '气促', '加重', '周'],
 ['咳嗽', '咳痰', '周', '加重', '伴', '发热', '天'],
 ['反复', '腹泻'],
 ['反复', '咳嗽', '咳痰', '气促', '加重', '天'],
 ['体检', '发现', '甲状腺肿', '物', '周'],
 ['反复', '咳嗽', '咳痰', '气促',

In [10]:
dictionary = corpora.Dictionary(corpus)
print(dictionary)

Dictionary(339 unique tokens: ['加重', '反复', '咳嗽', '咳痰', '天']...)


In [11]:
doc_vectors = [dictionary.doc2bow(text) for text in corpus]
print(len(doc_vectors))
print(doc_vectors)

1768
[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)], [(2, 1), (5, 1), (6, 1), (7, 1)], [(0, 1), (2, 1), (3, 1), (5, 1), (7, 1)], [(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (8, 1)], [(9, 1), (10, 1), (11, 1)], [(0, 1), (2, 1), (4, 1), (5, 1), (6, 1), (8, 1), (12, 1)], [(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (8, 1)], [(0, 1), (2, 1), (3, 1), (5, 1)], [(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)], [(13, 1)], [(1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (8, 1), (12, 1), (14, 1)], [(5, 1), (15, 1)], [(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)], [(0, 1), (1, 1), (2, 1), (7, 1), (16, 1)], [(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (8, 1)], [(1, 1), (5, 1), (10, 1), (14, 1)], [(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)], [(0, 1), (1, 1), (2, 1), (3, 1), (5, 1)], [(0, 1), (5, 1), (7, 1), (10, 1), (17, 1)], [(0, 1), (5, 1), (7, 1), (10, 1), (17, 1)], [(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)], [(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)], [(1, 1),

In [12]:
tfidf = models.TfidfModel(doc_vectors)
tfidf_vectors = tfidf[doc_vectors]
print(len(tfidf_vectors))
print((tfidf_vectors[0]))

1768
[(0, 0.37554419300874026), (1, 0.3117489963190072), (2, 0.25539982472619394), (3, 0.37615047917364675), (4, 0.6644420107645572), (5, 0.3370128829435504)]


In [13]:
#相似度矩阵
index = similarities.MatrixSimilarity(tfidf_vectors)
print(index)

MatrixSimilarity<1768 docs, 339 features>


D:\program\Anaconda\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [14]:
sims_matrix = []
for bow in doc_vectors:
    #单个短文本的词袋序列
    sim_seq = index[bow]
    sims_matrix.append(sim_seq)
print(sims_matrix)

[array([0.9472579 , 0.13741785, 0.41538313, ..., 0.9045423 , 0.        ,
       0.10106748], dtype=float32), array([0.29620636, 0.8534483 , 0.6530734 , ..., 0.3963476 , 0.        ,
       0.03351415], dtype=float32), array([0.60110307, 0.4384407 , 0.83860266, ..., 0.8043237 , 0.        ,
       0.07412428], dtype=float32), array([0.8769903 , 0.1272242 , 0.38457003, ..., 0.8374435 , 0.        ,
       0.0935703 ], dtype=float32), array([0.        , 0.        , 0.        , ..., 0.        , 0.        ,
       0.13253321], dtype=float32), array([0.6169888 , 0.40182042, 0.27694753, ..., 0.48954076, 0.        ,
       0.02533431], dtype=float32), array([0.8769903 , 0.1272242 , 0.38457003, ..., 0.8374435 , 0.        ,
       0.0935703 ], dtype=float32), array([0.6720537 , 0.16830182, 0.50873834, ..., 0.8992612 , 0.        ,
       0.08287346], dtype=float32), array([0.9472579 , 0.13741785, 0.41538313, ..., 0.9045423 , 0.        ,
       0.10106748], dtype=float32), array([0., 0., 0., ..., 0.,

In [15]:
from Network_cluster import Net_Spectral_clustering,Net_Hierarchical_clustering,Net_Fast_Unfolding
import scipy
import scipy.cluster.hierarchy as sch
from scipy.cluster.vq import vq,kmeans,whiten

In [16]:
spectral_clustering_lists, label = Net_Spectral_clustering(pd.DataFrame(sims_matrix), nclusters=50, affinity_v='precomputed')
np.save("zhusu_text_spectral_clustering_lists.npy",spectral_clustering_lists)
np.save("zhusu_text_spectral_clustering_label.npy",label)
label

D:\program\Anaconda\lib\site-packages\sklearn\utils\validation.py:709: UserWarning: Array is not symmetric, and will be converted to symmetric by average with its transpose.
  warnings.warn("Array is not symmetric, and will be converted "
D:\program\Anaconda\lib\site-packages\sklearn\manifold\spectral_embedding_.py:234: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"
D:\program\Anaconda\lib\site-packages\sklearn\cluster\spectral.py:442: UserWarning: The spectral clustering API has changed. ``fit``now constructs an affinity matrix from data. To use a custom affinity matrix, set ``affinity=precomputed``.
  warnings.warn("The spectral clustering API has changed. ``fit``"


array([ 4,  0,  0, ...,  2,  1, 17])

In [17]:
spectral_clustering_lists = np.load("zhusu_text_spectral_clustering_lists.npy")
label = np.load("zhusu_text_spectral_clustering_label.npy")
spectral_clustering_lists

array([list([1, 2, 4, 5, 10, 13, 15, 22, 23, 25, 26, 28, 31, 32, 33, 35, 42, 43, 48, 49, 51, 52, 54, 56, 57, 58, 59, 62, 64, 65, 66, 68, 69, 70, 71, 74, 75, 78, 79, 80, 84, 85, 87, 91, 93, 96, 97, 98, 99, 100, 101, 102, 104, 106, 108, 112, 115, 121, 123, 124, 126, 127, 132, 133, 138, 141, 144, 147, 149, 150, 151, 154, 155, 158, 159, 161, 163, 164, 165, 167, 168, 170, 171, 172, 173, 174, 177, 180, 181, 182, 185, 186, 187, 188, 191, 192, 197, 200, 205, 206, 208, 209, 211, 212, 213, 214, 219, 223, 224, 227, 233, 235, 236, 239, 243, 244, 245, 247, 248, 249, 251, 254, 255, 256, 259, 260, 261, 262, 263, 265, 266, 268, 270, 271, 272, 273, 275, 277, 278, 282, 283, 284, 285, 286, 287, 288, 289, 290, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 305, 308, 309, 310, 311, 313, 314, 315, 316, 318, 319, 321, 322, 325, 328, 329, 332, 333, 334, 337, 340, 341, 342, 344, 346, 347, 348, 349, 350, 351, 355, 357, 361, 363, 367, 368, 369, 371, 372, 373, 384, 385, 387, 390, 393, 396, 397, 398, 403, 404, 